In [1]:
# importing dependencies
import pandas as pd
from ipyleaflet import Map, Marker, Heatmap, basemaps, LayerGroup, CircleMarker, LegendControl
import folium

In [2]:
# loading the dataset
df = pd.read_csv('Resources/volcano-events.csv')

In [3]:
# cleaning the data
    # selecting the columns of interest
columns_of_interest = ['Latitude', 'Longitude', 'VEI', 'Total Deaths', 'Total Damage ($Mil)', 'Total Houses Destroyed']
df_filtered = df[columns_of_interest]

    # drop rows with missing values to ensure data integrity for visualization
df_clean = df_filtered.dropna(subset=['Latitude', 'Longitude', 'VEI'])

    # filling missing values with 0 instead of NaN
df_clean.fillna({'Total Deaths': 0, 'Total Damage ($Mil)': 0, 'Total Houses Destroyed': 0}, inplace=True)

C:\Users\mcayt\AppData\Local\Temp\ipykernel_376\2015854451.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.fillna({'Total Deaths': 0, 'Total Damage ($Mil)': 0, 'Total Houses Destroyed': 0}, inplace=True)


In [4]:
# initial work on the map

    # creating the map and centering it
m = Map(center=(0, 0), zoom=2, basemap=basemaps.Esri.WorldImagery)

    # adding markers for volcanoes
for index, row in df_clean.iterrows():
    marker = Marker(location=(row['Latitude'], row['Longitude']), draggable=False)
    m.add_layer(marker)

    # data for heatmap
heatmap_data = df_clean[['Latitude', 'Longitude', 'VEI']].values.tolist()

    # creating a heatmap layer
heatmap_layer = Heatmap(locations=heatmap_data, radius=20, blur=20, max_zoom=17)

    # adding the heatmap layer on the map
m.add_layer(heatmap_layer)

# display the map
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [5]:
# work on data visualization

    # adjusting marker color per VEI
def get_marker_color(vei):
    if vei < 2:
        return "green"
    elif vei < 4:
        return "yellow"
    elif vei < 5:
        return "orange"
    else:
        return "red"

# adjusting marker size per deaths
def get_marker_size(deaths):
    if deaths < 10:
        return 5 
    elif deaths < 100:
        return 7
    else:
        return 10

# adding markers onto the map
for index, row in df_clean.iterrows():
    marker = CircleMarker(
        location=(row['Latitude'], row['Longitude']),
        radius=get_marker_size(row['Total Deaths']),
        color=get_marker_color(row['VEI']),
        fill_color=get_marker_color(row['VEI']),
        fill_opacity=0.8
    )
    m.add_layer(marker)

# adding a legend to the map
legend = LegendControl({"Low VEI (<2)": "green", "Medium VEI (2-3)": "yellow", "High VEI (4)": "orange", "Very High VEI (>4)": "red"}, name="VEI Scale", position="bottomright")
m.add_control(legend)

# display the map
m


Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…